## This notebook merges results from the sourmash-gather output into one sheet with results from all samples. It also summarizes the fraction of base pairs unassigned and assigned to the genus Pelagibacter by sourmash.

#### Import all required modules

In [1]:
import pandas as pd
import os as os
import glob as glob
import numpy as np

#### Navigate to your working directory, which should contain both spreadsheets you want to merge

In [47]:
os.chdir("/Users/nastassia.patin/Desktop/Projects/Lasker2019/Long read paper/mSystems/Revisions/new-sourmash/nts-GTDB-MMETSP/tax_out")

#### Import an example sheet

In [59]:
summary = pd.read_csv("1903c117_50m-1_tax.summarized.csv")
summary.head()

,query_name,rank,fraction,lineage,query_md5,query_filename,f_weighted_at_rank,bp_match_at_rank
0,NaN,superkingdom,0.057079,d__Bacteria,41f6a87e,1903c117_50m-1_bmtag_interleaved.fq.gz,0.057079,35107000
1,NaN,superkingdom,0.014560,Eukaryota,41f6a87e,1903c117_50m-1_bmtag_interleaved.fq.gz,0.014560,8955000
2,NaN,superkingdom,0.014186,d__Archaea,41f6a87e,1903c117_50m-1_bmtag_interleaved.fq.gz,0.014186,8725000
3,NaN,superkingdom,0.914176,unclassified,41f6a87e,1903c117_50m-1_bmtag_interleaved.fq.gz,0.914176,1999000
4,NaN,phylum,0.037757,d__Bacteria;p__Proteobacteria,41f6a87e,1903c117_50m-1_bmtag_interleaved.fq.gz,0.037757,23223000


In [17]:
species = summary[summary['rank'] == 'species']
species = species[['fraction', 'lineage']]

In [18]:
species

,fraction,lineage
239,0.006619,Eukaryota;Viridiplantae:Chlorophyta;Prasinophy...
240,0.003585,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
241,0.002694,d__Archaea;p__Thermoplasmatota;c__Poseidoniia;...
242,0.002675,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
243,0.002379,d__Archaea;p__Thermoplasmatota;c__Poseidoniia;...
...,...,...
406,0.000081,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
407,0.000081,d__Bacteria;p__Marinisomatota;c__Marinisomatia...
408,0.000081,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...
409,0.000081,Eukaryota;Alveolata:Dinophyta;Dinophyceae;Sues...


## 1. Summarize and combine all sourmash tax summaries

### Function to reformat individual data sheet

In [9]:
def format_sourmash_summary(sheet, sample):
    taxon = sheet[sheet['rank'] == 'species'] # select desired level of taxonomic resolution
    taxon = taxon[['bp_match_at_rank', 'lineage']]
    taxon2 = taxon.groupby(['lineage']).sum()
    taxon2['sample'] = sample
    return(taxon2)

### Apply function to all data sheets and combine

In [10]:
summary = []

for file in glob.glob("*summarized.csv"):
    sheet = pd.read_csv(file)
    a, b, c = file.split('_')
    sample = a + "_" + b
    df = format_sourmash_summary(sheet, sample)
    summary.append(df)

summary = pd.concat(summary)

In [11]:
summary.head()

,bp_match_at_rank,sample
lineage,,
Eukaryota;Cryptophyta:Cryptophyta;Cryptophyceae;Pyrenomonadales;Geminigeracea;Geminigera;Geminigera sp.,54000,1903c117_50m-2
Eukaryota;Haptophyta:Haptophyta;Prymnesiophyceae;Isochrysidales;Isochrysidaceae;Isochrysis;Isochrysis sp.,119000,1903c117_50m-2
Eukaryota;Haptophyta:Haptophyta;Prymnesiophyceae;Isochrysidales;Noelaerhabdaceae;Emiliania;Emiliania huxleyi,378000,1903c117_50m-2
Eukaryota;Haptophyta:Haptophyta;Prymnesiophyceae;Phaeocystales;Phaeocystaceae;Phaeocystis;Phaeocystis antarctica,59000,1903c117_50m-2
Eukaryota;Haptophyta:Haptophyta;Prymnesiophyceae;Phaeocystales;Phaeocystacear;Phaeocystis;Phaeocystis sp.,210000,1903c117_50m-2


#### Pivot table so each sample is a column

In [12]:
summary_pivoted = summary.pivot_table(index='lineage', columns='sample', values='bp_match_at_rank')
summary_pivoted = summary_pivoted.fillna(0)
summary_pivoted.head()

sample,1903c111_10m-1,1903c111_10m-2,1903c111_10m-3,1903c117_50m-1,1903c117_50m-2,1903c117_50m-3,1903c118_23m-2,1903c118_23m-3,1903c119_11m-2,1903c119_11m-3,...,1903c144_13m-2,1903c144_13m-3,Las19c107_10m-1,Las19c107_10m-2,Las19c107_10m-3,Las19c135_5m-1,Las19c135_5m-2,Las19c135_5m-3,Las19c138_27m-1,Las19c138_27m-3
lineage,,,,,,,,,,,,,,,,,,,,,
Eukaryota;Alveolata:Ciliophora;Litostomatea;Cyclotrichiida;Mesodiniidae;Mesodinium;Mesodinium pulex,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0,0.0
Eukaryota;Alveolata:Ciliophora;Spirotrichea;Tintinnida;Ptychocylididae;Favella;Favella ehrenbergii,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,157000.0,0.0,0.0,0.0,0.0,206000.0,0.0,0.0,0.0,0.0
Eukaryota;Alveolata:Dinophyta;Dinophyceae;Gonyualacales;Crypthecodiniacea;Crypthecodinium;Crypthecodinium cohnii,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,60000.0,0.0,56000.0,0.0
Eukaryota;Alveolata:Dinophyta;Dinophyceae;Gonyualacales;Goniodomataceae;Alexandrium;Alexandrium tamarense,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,109000.0,0.0
Eukaryota;Alveolata:Dinophyta;Dinophyceae;Suessiales;Suessiaceae;Pelagodinium;Pelagodinium bii,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52000.0


#### Save the merged table as a csv

In [13]:
summary_pivoted.to_csv("Lasker2019_Illumina_PacBio_sourmash_GTDB_MMETSP_species_bpnumbers.csv")

## 2. Summarize % unclassified reads and % assigned to genus Pelagibacter

In [43]:
def extract_fraction_unassigned(sheet, sample):
    taxon = sheet[sheet['rank'] == 'species'] # select desired level of taxonomic resolution
    taxon = taxon[['fraction', 'lineage']]
    unassigned = taxon.loc[taxon['lineage'] == 'unclassified']
    unassigned.insert(len(unassigned.columns), 'sample', sample)
    return(unassigned)

In [56]:
def extract_fraction_pelagibacter(sheet, sample):
    taxon = sheet[sheet['rank'] == 'genus'] # select desired level of taxonomic resolution
    taxon = taxon[['fraction', 'lineage']]
    SAR11 = taxon.loc[taxon['lineage'].str.contains('Pelagibacter')]
    SAR11.insert(len(SAR11.columns), 'sample', sample)
    return(SAR11)

#### Summarize % unclassified

In [62]:
summary = []

for file in glob.glob("*summarized.csv"):
    sheet = pd.read_csv(file)
    a, b, c = file.split('_')
    sample = a + "_" + b
    df = extract_fraction_unassigned(sheet, sample)
    summary.append(df)

frac_unassigned = pd.concat(summary)

#### Summarize % Pelagibacter

In [75]:
summary = []

for file in glob.glob("*summarized.csv"):
    sheet = pd.read_csv(file)
    a, b, c = file.split('_')
    sample = a + "_" + b
    df = extract_fraction_pelagibacter(sheet, sample)
    df = df.groupby('sample').sum() # there are couple different Pelagibacter lineages
    df.insert(len(df.columns), 'Genus', 'Pelagibacter') 
    summary.append(df)

frac_pelagibac = pd.concat(summary)

In [76]:
frac_pelagibac

,fraction,Genus
sample,,
1903c117_50m-2,0.000428,Pelagibacter
1903c124_15m-3,0.021033,Pelagibacter
1903c118_23m-2,0.002269,Pelagibacter
1903c129_26m-2,0.000129,Pelagibacter
1903c129_26m-1,0.001326,Pelagibacter
1903c117_50m-1,0.000834,Pelagibacter
1903c127_7m-1,0.001576,Pelagibacter
Las19c107_10m-2,0.000361,Pelagibacter
1903c144_13m-2,0.002834,Pelagibacter


In [71]:
frac_unassigned.to_csv("Lasker2019_Illumina_PacBio_sourmash_GTDB_MMETSP_fractionunassigned.csv", index=None)

In [78]:
frac_pelagibac.to_csv("Lasker2019_Illumina_PacBio_sourmash_GTDB_MMETSP_fractionSAR11.csv")